# Information Retrieval Lab WiSe 2024/2025: Baseline Retrieval System

This Jupyter notebook serves as a baseline retrieval system that you can improve upon.
We use subsets of the MS MARCO datasets to retrieve passages of web documents.
We will show you how to create a software submission to TIRA from this notebook.

An overview of all corpora that we use in the current course is available at [https://tira.io/datasets?query=ir-lab-wise-2024](https://tira.io/datasets?query=ir-lab-wise-2024). The dataset IDs for loading the datasets are:

- `ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training`: A subsample of the TREC 2019/2020 Deep Learning tracks on the MS MARCO v1 passage dataset. Use this dataset to tune your system(s).
- `ir-lab-wise-2024/subsampled-ms-marco-rag-20241202-training` (_work in progress_): A subsample of the TREC 2024 Retrieval-Augmented Generation track on the MS MARCO v2.1 passage dataset. Use this dataset to tune your system(s).
- `ir-lab-wise-2024/ms-marco-rag-20241203-test` (work in progress): The test corpus that we have created together in the course, based on the MS MARCO v2.1 passage dataset. We will use this dataset as the test dataset, i.e., evaluation scores become available only after the submission deadline.

### Step 1: Import libraries

We will use [tira](https://tira.io/), an information retrieval shared task platform, and [ir_dataset](https://ir-datasets.com/) for loading the datasets. Subsequently, we will build a retrieval system with [PyTerrier](https://github.com/terrier-org/pyterrier), an open-source search engine framework.

First, we need to install the required libraries.

In [1]:
!pip3 install 'tira>=0.0.139' ir-datasets 'python-terrier==0.10.0'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.9/112.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.9/347.9 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 18.8 MB/s eta 0:00:00
  

Create an API client to interact with the TIRA platform (e.g., to load datasets and submit runs).

In [2]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded
from tira.rest_api_client import Client
import pyterrier as pt

ensure_pyterrier_is_loaded()
tira = Client()

terrier-assemblies 5.7 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.7 jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.0 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



### Step 2: Load the dataset

We load the dataset by its ir_datasets ID (as listed in the Readme). Just be sure to add the `irds:` prefix before the dataset ID to tell PyTerrier to load the data from ir_datasets.

In [3]:
from pyterrier import get_dataset

pt_dataset = get_dataset('irds:ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training')

### Step 3: Build an index

We will then create an index from the documents in the dataset we just loaded.

In [4]:
from pyterrier import IterDictIndexer

indexer = IterDictIndexer(
    # Store the index in the `index` directory.
    "../data/index",
    meta={'docno': 50, 'text': 4096},
    # If an index already exists there, then overwrite it.
    overwrite=True,
)
index = indexer.index(pt_dataset.get_corpus_iter())

Download from Zenodo: https://zenodo.org/records/14254044/files/subsampled-ms-marco-deep-learning-20241201-training-inputs.zip


Download: 100%|██████████| 9.51M/9.51M [00:02<00:00, 4.09MiB/s]


Download finished. Extract...
Extraction finished:  /root/.tira/extracted_datasets/ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training/


ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents:   0%|          | 0/68261 [00:0…

20:09:46.192 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (6114613) - further warnings are suppressed
20:10:01.061 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 1 empty documents


### Step 4: Define the retrieval pipeline

We will define a simple retrieval pipeline using just BM25 as a baseline. For details, refer to the PyTerrier [documentation](https://pyterrier.readthedocs.io) or [tutorial](https://github.com/terrier-org/ecir2021tutorial).

In [5]:
from pyterrier import BatchRetrieve

bm25 = BatchRetrieve(index, wmodel="BM25")

### Step 5: Create the run
In the next steps, we would like to apply our retrieval system to some topics, to prepare a 'run' file, containing the retrieved documents.

First, let's have a short look at the first three topics:

In [6]:
# The `'text'` argument below selects the topics `text` field as the query.
df = pt_dataset.get_topics('text')
pt_dataset.get_topics('text')

Download from Zenodo: https://zenodo.org/records/14254044/files/subsampled-ms-marco-deep-learning-20241201-training-truths.zip


Download: 100%|██████████| 61.7k/61.7k [00:00<00:00, 131kiB/s] 

Download finished. Extract...
Extraction finished:  /root/.tira/extracted_datasets/ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training/


,qid,query
0,1030303,who is aziz hashim
1,1037496,who is rep scalise
2,1043135,who killed nicholas ii of russia
3,1051399,who sings monk theme song
4,1064670,why do hunters pattern their shotguns
...,...,...
92,405717,is cdg airport in main paris
93,182539,example of monotonic function
94,1113437,what is physical description of spruce
95,1129237,hydrogen is a liquid below what temperature


Now, retrieve results for all the topics (may take a while):

In [7]:
run = bm25(pt_dataset.get_topics('text'))

That's it for the retrieval. Here are the first 10 entries of the run:

In [8]:
run.head(10)

,qid,docid,docno,rank,score,query
0,1030303,53852,8726436,0,31.681671,who is aziz hashim
1,1030303,56041,8726433,1,25.966276,who is aziz hashim
2,1030303,62116,8726435,2,23.863442,who is aziz hashim
3,1030303,32183,8726429,3,23.391821,who is aziz hashim
4,1030303,35867,8726437,4,21.030669,who is aziz hashim
5,1030303,17637,8726430,5,19.967200,who is aziz hashim
6,1030303,42957,7156982,6,19.967200,who is aziz hashim
7,1030303,21803,8726434,7,19.474804,who is aziz hashim
8,1030303,59828,1305520,8,17.849161,who is aziz hashim
9,1030303,60002,3302257,9,17.832781,who is aziz hashim


### Step 6: Evaluate your run

In [9]:
from pyterrier import Experiment

Experiment([bm25],
    pt_dataset.get_topics('text'),
    pt_dataset.get_qrels(),
    eval_metrics = ["map", "recip_rank", "ndcg_cut_10", "P_1", "P_5", "P_10"]
)

,name,map,recip_rank,ndcg_cut_10,P_1,P_5,P_10
0,BR(BM25),0.412718,0.786653,0.489469,0.701031,0.62268,0.574227


### Step 7: Improve

Building your own index can be already one way that you can try to improve upon this baseline (if you want to focus on creating good document representations). Other ways could include reformulating queries or tuning parameters or building better retrieval pipelines.

# Data Cleaning and Preprocessing

In [10]:
def preprocess_text(text):
    # Comprehensive text cleaning pipeline
    text = text.lower()  # Lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers

    # Advanced cleaning steps
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace

    return text

# Stop word removal
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Comparison of multiple retrieval models with parameter tuning (BM25 + TF_IDF)

In [11]:
# Multiple retrieval models with parameter tuning

# BM25 with varying k1 and b values
# k1: Controls the term frequency saturation. A higher value means more weight on term frequency.
# b: Controls the length normalization. Values closer to 1 mean greater length normalization.
bm25_tuned = [
    BatchRetrieve(index, wmodel="BM25", controls={"c": k1, "b": b})
    for k1 in [0.5, 1.0, 1.5]
    for b in [0.2, 0.5, 0.8]
]

# TF-IDF model
tfidf = BatchRetrieve(index, wmodel="TF_IDF")

In [12]:
results = Experiment(
    retr_systems=[bm25, tfidf] + bm25_tuned,
    topics=pt_dataset.get_topics('text'),
    qrels=pt_dataset.get_qrels(),
    eval_metrics=["map", "recip_rank", "ndcg_cut_10", "P_1", "P_5", "P_10"]
)

print(results)

          name       map  recip_rank  ndcg_cut_10       P_1       P_5  \
0     BR(BM25)  0.412718    0.786653     0.489469  0.701031  0.622680   
1   BR(TF_IDF)  0.410917    0.786086     0.488676  0.701031  0.620619   
2     BR(BM25)  0.413929    0.788569     0.491176  0.701031  0.637113   
3     BR(BM25)  0.413929    0.788569     0.491176  0.701031  0.637113   
4     BR(BM25)  0.413929    0.788569     0.491176  0.701031  0.637113   
5     BR(BM25)  0.401502    0.790825     0.470550  0.701031  0.616495   
6     BR(BM25)  0.401502    0.790825     0.470550  0.701031  0.616495   
7     BR(BM25)  0.401502    0.790825     0.470550  0.701031  0.616495   
8     BR(BM25)  0.347434    0.649607     0.373324  0.494845  0.496907   
9     BR(BM25)  0.347434    0.649607     0.373324  0.494845  0.496907   
10    BR(BM25)  0.347434    0.649607     0.373324  0.494845  0.496907   

        P_10  
0   0.574227  
1   0.572165  
2   0.575258  
3   0.575258  
4   0.575258  
5   0.545361  
6   0.545361  
7  

# Try out more models

In [15]:
# BM25F: Field-based BM25 model for multi-field indexing
# PL2: A probabilistic retrieval model based on the Divergence From Randomness (DFR) framework
# DirichletLM: A language modeling approach with Dirichlet smoothing

# BM25F (Field-based BM25)
bm25f_tuned = [
    (BatchRetrieve(index, wmodel="BM25F", controls={"k1": k1, "b": b}), f"BM25F(k1={k1}, b={b})")
    for k1 in [0.5, 1.0, 1.5]
    for b in [0.2, 0.5, 0.8]
]

# PL2 (DFR model)
pl2_tuned = [
    (BatchRetrieve(index, wmodel="PL2", controls={"c": c}), f"PL2(c={c})")
    for c in [0.1, 0.5, 1.0]
]

# DirichletLM (Language model with Dirichlet smoothing)
dirichletlm_tuned = [
    (BatchRetrieve(index, wmodel="DirichletLM", controls={"mu": mu}), f"DirichletLM(mu={mu})")
    for mu in [500, 1000, 1500]
]

# Define retrievers with labels
retrievers = bm25f_tuned + pl2_tuned + dirichletlm_tuned

# Extract only the retrieval systems
retr_systems_only = [retriever for retriever, label in retrievers]

In [16]:
# Run the experiment
results = Experiment(
    retr_systems=retr_systems_only,
    topics=pt_dataset.get_topics('text'),
    qrels=pt_dataset.get_qrels(),
    eval_metrics=["map", "recip_rank", "ndcg_cut_10", "P_1", "P_5", "P_10"]
)

# Add labels back to the results
results['System'] = [label for _, label in retrievers]

print(results)

               name       map  recip_rank  ndcg_cut_10       P_1       P_5  \
0         BR(BM25F)  0.401502    0.790825     0.470550  0.701031  0.616495   
1         BR(BM25F)  0.401502    0.790825     0.470550  0.701031  0.616495   
2         BR(BM25F)  0.401502    0.790825     0.470550  0.701031  0.616495   
3         BR(BM25F)  0.401502    0.790825     0.470550  0.701031  0.616495   
4         BR(BM25F)  0.401502    0.790825     0.470550  0.701031  0.616495   
5         BR(BM25F)  0.401502    0.790825     0.470550  0.701031  0.616495   
6         BR(BM25F)  0.401502    0.790825     0.470550  0.701031  0.616495   
7         BR(BM25F)  0.401502    0.790825     0.470550  0.701031  0.616495   
8         BR(BM25F)  0.401502    0.790825     0.470550  0.701031  0.616495   
9           BR(PL2)  0.227536    0.631489     0.293182  0.505155  0.412371   
10          BR(PL2)  0.358087    0.731683     0.432186  0.618557  0.581443   
11          BR(PL2)  0.393056    0.775629     0.469707  0.690722